## Convolutional Neural Network
---
Complete flow of CNN to process an input image and classifies the objects based on values.

![CNN Overview](images/cnn.jpeg)

### The Road Ahead 

* [Step 1](#Step1): Import Datasets
* [Step 2](#Step2): Data Preprocessing
* [Step 3](#Step3): Model Architecture
* [Step 4](#Step4): Compiling Model
* [Step 5](#Step5): Algorithm To Predict Cancer

----------------------------------
<a id = 'Step1'></a>
## Step 1: Import Datasets


In [1]:
# Import modules that is needed
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob


Using TensorFlow backend.


In [2]:
# define function to Load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    data_files = np.array(data['filenames'])
    data_targets = np_utils.to_categorical(data['target'], 3)
    return data_files, data_targets


In [3]:
test_files, test_targets = load_dataset('test')

In [4]:
# Load train, test, and validation datasets
train_files, train_targets = load_dataset('train')
valid_files, valid_targets = load_dataset('valid')

In [4]:
# Load list of class name
class_names = [item[6:-1] for item in sorted(glob("train/*/"))]

# Statistics about dataset
print('There are total', len(class_names), 'categories.')
print('There are total', len(np.hstack([train_files, test_files, valid_files])), 'images of skin.\n')
print('Tere are total', len(train_files), 'images for training.')
print('Tere are total', len(test_files), 'images for testing.')
print('Tere are total', len(valid_files), 'images for validation.')

There are total 3 categories.


NameError: name 'train_files' is not defined

---
<a id = 'Step2'></a>
## Step 2: Data Preprocessing
Conveting Images to RGB matrix.

![RGB Image](images/rgb.png)

In [5]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(299, 299))
    # convert PIL.Image.Image type to 3D tensor with shape (299, 299, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 299, 299, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [7]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [05:46<00:00,  1.71it/s]


In [6]:
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [05:31<00:00,  1.70it/s]


---
<a id="Step3"></a>
## Step 3: Model Architecture

CNN model architecture using keras.

![Sample CNN](images/sample_cnn.png)

In [8]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=16, 
                 kernel_size=2, 
                 padding='same', 
                 activation='relu',
                 input_shape=(299, 299, 3)
                ))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,
                 kernel_size=2,
                 padding='same',
                 activation='relu',
                ))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,
                 kernel_size=2,
                 padding='same',
                 activation='relu'
                ))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128,
                 kernel_size=2,
                 padding='same',
                 activation='relu'
                ))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 299, 299, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 149, 149, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxP

<a id = "Step4"></a>
## Step 4: Compiling Model

In [9]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
from keras.callbacks import ModelCheckpoint  

epochs = 10

checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/10
2000/2000 [==============================] - ETA: 10:09 - loss: 0.7089 - acc: 0.70 - ETA: 5:14 - loss: 0.6853 - acc: 0.7250 - ETA: 3:36 - loss: 0.6180 - acc: 0.766 - ETA: 2:46 - loss: 0.5953 - acc: 0.775 - ETA: 2:16 - loss: 0.6576 - acc: 0.740 - ETA: 1:56 - loss: 0.6306 - acc: 0.758 - ETA: 1:42 - loss: 0.6270 - acc: 0.757 - ETA: 1:31 - loss: 0.6073 - acc: 0.768 - ETA: 1:23 - loss: 0.5986 - acc: 0.777 - ETA: 1:16 - loss: 0.5946 - acc: 0.780 - ETA: 1:11 - loss: 0.6070 - acc: 0.777 - ETA: 1:06 - loss: 0.5973 - acc: 0.779 - ETA: 1:02 - loss: 0.5847 - acc: 0.780 - ETA: 58s - loss: 0.5752 - acc: 0.782 - ETA: 55s - loss: 0.6155 - acc: 0.77 - ETA: 52s - loss: 0.6132 - acc: 0.76 - ETA: 50s - loss: 0.6150 - acc: 0.77 - ETA: 51s - loss: 0.6022 - acc: 0.77 - ETA: 49s - loss: 0.5916 - acc: 0.77 - ETA: 47s - loss: 0.6021 - acc: 0.77 - ETA: 45s - loss: 0.6061 - acc: 0.76 - ETA: 43s - loss: 0.5967 - acc: 0.76 - ETA: 42s - loss: 0.5869 - acc: 0.

2000/2000 [==============================] - ETA: 16s - loss: 0.4137 - acc: 0.90 - ETA: 16s - loss: 0.4894 - acc: 0.80 - ETA: 16s - loss: 0.4793 - acc: 0.76 - ETA: 16s - loss: 0.4307 - acc: 0.80 - ETA: 16s - loss: 0.4064 - acc: 0.80 - ETA: 15s - loss: 0.4289 - acc: 0.78 - ETA: 15s - loss: 0.4312 - acc: 0.80 - ETA: 15s - loss: 0.4522 - acc: 0.78 - ETA: 15s - loss: 0.4670 - acc: 0.79 - ETA: 15s - loss: 0.4675 - acc: 0.78 - ETA: 14s - loss: 0.4702 - acc: 0.78 - ETA: 14s - loss: 0.4528 - acc: 0.80 - ETA: 14s - loss: 0.4595 - acc: 0.80 - ETA: 14s - loss: 0.4810 - acc: 0.78 - ETA: 14s - loss: 0.4843 - acc: 0.78 - ETA: 14s - loss: 0.4723 - acc: 0.78 - ETA: 13s - loss: 0.4678 - acc: 0.79 - ETA: 13s - loss: 0.4551 - acc: 0.80 - ETA: 13s - loss: 0.4564 - acc: 0.79 - ETA: 13s - loss: 0.4959 - acc: 0.79 - ETA: 13s - loss: 0.4888 - acc: 0.79 - ETA: 13s - loss: 0.5117 - acc: 0.78 - ETA: 12s - loss: 0.5085 - acc: 0.78 - ETA: 12s - loss: 0.5057 - acc: 0.78 - ETA: 12s - loss: 0.5035 - acc: 0.79 - ETA: 

2000/2000 [==============================] - ETA: 16s - loss: 0.2413 - acc: 0.90 - ETA: 17s - loss: 0.2918 - acc: 0.87 - ETA: 16s - loss: 0.2740 - acc: 0.86 - ETA: 16s - loss: 0.3195 - acc: 0.85 - ETA: 16s - loss: 0.3250 - acc: 0.86 - ETA: 16s - loss: 0.3519 - acc: 0.84 - ETA: 15s - loss: 0.3832 - acc: 0.84 - ETA: 15s - loss: 0.3809 - acc: 0.83 - ETA: 15s - loss: 0.3795 - acc: 0.83 - ETA: 15s - loss: 0.3767 - acc: 0.84 - ETA: 15s - loss: 0.3707 - acc: 0.84 - ETA: 14s - loss: 0.3519 - acc: 0.85 - ETA: 14s - loss: 0.3495 - acc: 0.85 - ETA: 14s - loss: 0.3591 - acc: 0.85 - ETA: 14s - loss: 0.3595 - acc: 0.84 - ETA: 14s - loss: 0.3520 - acc: 0.85 - ETA: 13s - loss: 0.3391 - acc: 0.86 - ETA: 13s - loss: 0.3376 - acc: 0.86 - ETA: 13s - loss: 0.3291 - acc: 0.86 - ETA: 13s - loss: 0.3205 - acc: 0.87 - ETA: 13s - loss: 0.3232 - acc: 0.86 - ETA: 13s - loss: 0.3159 - acc: 0.87 - ETA: 12s - loss: 0.3218 - acc: 0.86 - ETA: 12s - loss: 0.3341 - acc: 0.85 - ETA: 12s - loss: 0.3390 - acc: 0.85 - ETA: 

In [10]:
model.load_weights('weights.best.from_scratch.hdf5')

In [12]:
# get index of predicted skin for each image in test set
skin_cancer_predictions = [np.argmax(model.predict(np.expand_dims(test_tensors[0], axis=0))) for tensor in test_tensors]

# Test accuracy
test_accuracy = 100*np.sum(np.array(skin_cancer_predictions)==np.argmax(test_targets, axis=1))/len(skin_cancer_predictions)

ValueError: unsupported format character '%' (0x25) at index 18

In [13]:
a = path_to_tensor('test_images\\2.jpg').astype('float32')/255

In [16]:
print('Test accuracy: %.3f%%' %test_accuracy)

Test accuracy: 65.500%


---
<a id = "Step5"></a>
## Step 5: Algorithm To Predict Cancer

In [12]:
def predictCancer(imgpath):
    """
    Attributes
    ----------
    imgpath: string
            Contains the address of image.
    """
    
    data = path_to_tensor(imgpath).astype('float32')/255
    result = model.predict(data)
    print(result)
    print(class_names[np.argmax(result)])
    

In [14]:
path = input("Enter image path\n")
predictCancer("test_images\\" + path + ".jpg")
model.predict(path_to_tensor('test_images\\4.jpg').astype('float32')/255)

Enter image path
2
[[0.27490517 0.681469   0.04362569]]
nevus


array([[5.8864327e-18, 1.0000000e+00, 3.2709350e-26]], dtype=float32)

![Test Image](test_images\\2.jpg)